In [4]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://download.pytorch.org/whl/cu118


In [42]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import pandas as pd
import numpy as np
import re

In [8]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')
model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

2025-03-31 11:49:04.484604: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-31 11:49:04.644845: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1743401944.701415   25765 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1743401944.717442   25765 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1743401944.835844   25765 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [14]:
tokens = tokenizer.encode('It was good but couldve been better. Great', return_tensors='pt')

In [19]:
tokens[0]

tensor([  101, 10197, 10140, 12050, 10502, 12296, 10598, 10662, 16197,   119,
        11838,   102])

In [20]:
tokenizer.decode(tokens[0])

'[CLS] it was good but couldve been better. great [SEP]'

In [15]:
result = model(tokens)

the output from the model is a one-hot encoded list of scores  
the position with the highest score represents the sentiment rating

In [22]:
result

SequenceClassifierOutput(loss=None, logits=tensor([[-2.7768, -1.2353,  1.4419,  1.9804,  0.4584]],
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

the logits tensor is the direct output of the last linear layer of the model  
each number corresponds to the model’s confidence score for that class  
higher values indicate stronger confidence for that class

In [16]:
result.logits

tensor([[-2.7768, -1.2353,  1.4419,  1.9804,  0.4584]],
       grad_fn=<AddmmBackward0>)

In [25]:
torch.argmax(result.logits)

tensor(3)

In [23]:
int(torch.argmax(result.logits))+1

4

In [44]:
df = pd.read_csv('reddit_data_not_processed.csv')

In [48]:
titles = df[['title']].copy()

In [59]:
def sentiment_score(title):
    tokens = tokenizer.encode(title, return_tensors='pt')
    result = model(tokens)
    return int(torch.argmax(result.logits))

In [60]:
sentiment_score(df['title'].iloc[1])

0

In [61]:
df['sentiment'] = df['title'].apply(lambda x: sentiment_score(x[:100]))

In [62]:
print(df[['title', 'sentiment']].head(20))

                                                title  sentiment
0                      Does everyone fuck up in life?          0
1                                 I Wana kill my self          0
2                                          Struggling          0
3   I have lost all of my close friends. Don't kno...          1
4                              It’s creeping in again          2
5          Physical symptoms of a depressive episode?          0
6                Damned if you do damned if you don't          0
7   I am reaching my limit, I cant even find peace...          4
8                         How much more could I take?          1
9                           I don't understand myself          1
10             Reminder-Brush your teeth its worth it          3
11                                              Tired          0
12  Asperger with bipolar who feels great again an...          4
13                   There is something wrong with me          0
14                       

In [58]:
print(model.config.id2label)

{0: '1 star', 1: '2 stars', 2: '3 stars', 3: '4 stars', 4: '5 stars'}
